In [1]:
# import the required packages
import numpy as np
import matplotlib.pyplot as plt
import psrf

In [50]:
#initialize global parameters
x = np.array([-0.86, -0.30, -0.05, 0.73])
n = 5
y = np.array([0, 1, 3, 5])

#config
plt.rc('font', size=12)

rnorm = np.random.normal
runif = np.random.rand

In [51]:
#defining the method to get log posterior
def bioassaylp(a, b, x=x, y=y, n=n):
    t = a + b * x
    et = np.exp(t)
    z = et / (1. + et)
    lp = np.sum(y * np.log(z) + (n - y) * np.log(1. - z))
    return (lp)

In [27]:
# defining the method for metropilis algorithm
def metropolis(n_iterations, initial_values, prop_var=1):
    # np.random.seed(seed=1)
    n_params = len(initial_values)

    # Initial proposal standard deviations
    prop_sd = [prop_var] * n_params # [1,1]
    # Initialize trace for parameters
    trace = np.empty((n_iterations + 1, n_params))
    # Set initial values
    trace[0] = initial_values

    # Calculate joint posterior for initial values
    current_prob = bioassaylp(trace[0][0], trace[0][1])
    
    # Initialize acceptance counts
    accepted = [0] * n_params #[0,0]
    for i in range(n_iterations):
        # Grab current parameter values
        current_params = trace[i]

        # Get current value for parameter j
        p = trace[i].copy()
        for j in range(n_params):
            theta = rnorm(current_params[j], prop_sd[j])

            # Insert new value
            p[j] = theta

            # Calculate log posterior with proposed value
            proposed_prob = bioassaylp(*p) # *p is the spreading operator

            # Log-acceptance rate
            # taking the ratio of the probabilities is equal to minus the log of probabilities
            alpha = np.exp(proposed_prob - current_prob)

            # Sample a uniform random variate
            u = runif()

            # Test proposed value
            if u < alpha:
                # Accept
                trace[i + 1, j] = theta
                current_prob = proposed_prob
                accepted[j] += 1

            else:
                # Stay put
                trace[i + 1, j] = trace[i, j]

            p[j] = trace[i + 1, j]
    return trace, accepted

In [ ]:
n_iter = 10000
starting_points = [(2, 20), (4, 20), (-1, 15), (3, 12)]
warmuplen = 5000
traces_withoutwarmup = []

for x in starting_points:
    trace, acc = metropolis(n_iter, x, 1)
    alpha = trace[warmuplen:, 0];
    beta = trace[warmuplen:, 1];
    traces_withoutwarmup.append(np.column_stack((alpha, beta)))

stacked_traces = np.concatenate(traces_withoutwarmup)
print("Convergance Analysis for samples - PSRF values:", psrf.psrf(stacked_traces))
plt.scatter(stacked_traces[:, 0], stacked_traces[:, 1], s=0.4, alpha=0.6)
plt.ylim([-10, 40])
plt.xlim([-4, 10])
plt.ylabel(r'$\beta$')
plt.xlabel(r'$\alpha$')
plt.show()